In [ ]:
import os
import sys
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
csvpath = '/net/birdstore/Active_Atlas_Data/data_root/atlas_data/atlasV8/csv'

In [ ]:
df_midbrain_distance = pd.read_csv(os.path.join(csvpath, 'midbrain_distance.csv'))
df_midbrain_allen = pd.read_csv(os.path.join(csvpath, 'midbrain_allen.csv'))
df_midbrain_atlas = pd.read_csv(os.path.join(csvpath, 'midbrain_atlas.csv'))

In [ ]:
df_brainstem_distance = pd.read_csv(os.path.join(csvpath, 'brainstem_distance.csv'))
df_brainstem_allen = pd.read_csv(os.path.join(csvpath, 'brainstem_allen.csv'))
df_brainstem_atlas = pd.read_csv(os.path.join(csvpath, 'brainstem_atlas.csv'))

 That does not preclude starting the new idea but is just to say that somehow we need to get David's attention on what you have done- do a graph
= box plot of the midbrain com distances vs allen (active atlas based)
=box plot of the brainstem vs allen distances (but not using midbrain values to improve the distances)
This should provide an easy visual...


When the tethering is done for the brainstem structures though- he does not want the midbrain metrics to be included ( I don't know though if he means that the alignment is done with midbrain and then the evaluated outputs are only for the brainstem structures...?) your guess is best
So- in any case it was clear that he did not want metrics for midbrain and brainstem structures to be combined.

In [ ]:
midbrain_distances = df_midbrain_distance.distance
brainstem_distances = df_brainstem_distance.distance
fig, ax = plt.subplots()
columns = [midbrain_distances, brainstem_distances]
ax.boxplot(columns, patch_artist=True)
title1 = f"Midbrain distances\nn={len(midbrain_distances)}, min={round(min(midbrain_distances))}"
title1 += f"\nmax={round(max(midbrain_distances))}, mean={round(np.mean(midbrain_distances))}"
title2 = f"Brain stem distances\nn={len(brainstem_distances)}, min={round(min(brainstem_distances))}"
title2 += f"\nmax={round(max(brainstem_distances))}, mean={round(np.mean(brainstem_distances))}"
plt.xticks([1,2], [title1, title2])
plt.show()

In [ ]:
len(brainstem_distances), min(brainstem_distances), max(brainstem_distances), np.mean(brainstem_distances)

In [ ]:
for i, row in df_brainstem_distance.iterrows():
    print(f'<tr><td>{row[0]}</td> <td>{round(row[1]),2}</td></tr>')

In [ ]:
xmin = 8000
xmax = 13000
ymin = 2000
ymax = 6000
zmin = 0
zmax = 25*456

In [ ]:
for color, label in zip('bgr', ['Left', 'Right', 'Singular']):
    allen_subset = df_midbrain_allen[df_midbrain_allen.S == label]
    atlas_subset = df_midbrain_atlas[df_midbrain_atlas.S == label]
    plt.scatter(allen_subset.X, allen_subset.Y, s=120, marker='^', c=color, label=str('Allen ' + label))
    plt.scatter(atlas_subset.X, atlas_subset.Y, s=120, marker='o', c=color, label=str('Atlas ' + label))
    plt.xlabel('X')
    plt.ylabel('Y')
    ax = plt.gca()
    ax.set_xlim([xmin, xmax])
    ax.set_ylim([ymin, ymax])
    ax.invert_yaxis()
plt.title('Allen/Altas X Y scatter plot')    
plt.legend()

In [ ]:
for color, label in zip('bgr', ['Left', 'Right', 'Singular']):
    allen_subset = df_midbrain_allen[df_midbrain_allen.S == label]
    atlas_subset = df_midbrain_atlas[df_midbrain_atlas.S == label]
    plt.scatter(allen_subset.X, allen_subset.Z, s=120, marker='^', c=color, label=str('Allen ' + label))
    plt.scatter(atlas_subset.X, atlas_subset.Z, s=120, marker='o', c=color, label=str('Atlas ' + label))
    plt.xlabel('X')
    plt.ylabel('Z')
    ax = plt.gca()
    ax.set_xlim([xmin, xmax])
    #ax.set_ylim([ymin, ymax])
    #ax.invert_yaxis()
plt.title('Allen/Altas X Z scatter plot')    
plt.legend()

In [ ]:
for color, label in zip('bgr', ['Left', 'Right', 'Singular']):
    allen_subset = df_midbrain_allen[df_midbrain_allen.S == label]
    atlas_subset = df_midbrain_atlas[df_midbrain_atlas.S == label]
    plt.scatter(allen_subset.Z, allen_subset.Y, s=120, marker='^', c=color, label=str('Allen ' + label))
    plt.scatter(atlas_subset.Z, atlas_subset.Y, s=120, marker='o', c=color, label=str('Atlas ' + label))
    plt.xlabel('Z')
    plt.ylabel('Y')
    ax = plt.gca()
    #ax.set_xlim([xmin, xmax])
    #ax.set_ylim([ymin, ymax])
    ax.invert_yaxis()
plt.title('Allen/Altas Y Z scatter plot')    
plt.legend()

In [ ]:
for color, label in zip('bgr', ['Left', 'Right', 'Singular']):
    allen_subset = df_brainstem_allen[df_brainstem_allen.S == label]
    atlas_subset = df_brainstem_atlas[df_brainstem_atlas.S == label]
    plt.scatter(allen_subset.X, allen_subset.Y, s=120, marker='^', c=color, label=str('Allen ' + label))
    plt.scatter(atlas_subset.X, atlas_subset.Y, s=120, marker='o', c=color, label=str('Atlas ' + label))
    plt.xlabel('X')
    plt.ylabel('Y')
    ax = plt.gca()
    ax.set_xlim([xmin, xmax])
    ax.set_ylim([ymin, ymax])
    ax.invert_yaxis()
plt.title('Allen/Altas X Y scatter plot')    
plt.legend()

In [ ]:
for color, label in zip('bgr', ['Left', 'Right', 'Singular']):
    allen_subset = df_brainstem_allen[df_brainstem_allen.S == label]
    atlas_subset = df_brainstem_atlas[df_brainstem_atlas.S == label]
    plt.scatter(allen_subset.X, allen_subset.Z, s=120, marker='^', c=color, label=str('Allen ' + label))
    plt.scatter(atlas_subset.X, atlas_subset.Z, s=120, marker='o', c=color, label=str('Atlas ' + label))
    plt.xlabel('X')
    plt.ylabel('Z')
    ax = plt.gca()
    ax.set_xlim([xmin, xmax])
    #ax.set_ylim([ymin, ymax])
    #ax.invert_yaxis()
plt.title('Allen/Altas X Z scatter plot')    
plt.legend()

In [ ]:
for color, label in zip('bgr', ['Left', 'Right', 'Singular']):
    allen_subset = df_brainstem_allen[df_brainstem_allen.S == label]
    atlas_subset = df_brainstem_atlas[df_brainstem_atlas.S == label]
    plt.scatter(allen_subset.Z, allen_subset.Y, s=120, marker='^', c=color, label=str('Allen ' + label))
    plt.scatter(atlas_subset.Z, atlas_subset.Y, s=120, marker='o', c=color, label=str('Atlas ' + label))
    plt.xlabel('Z')
    plt.ylabel('Y')
    ax = plt.gca()
    #ax.set_xlim([xmin, xmax])
    #ax.set_ylim([ymin, ymax])
    ax.invert_yaxis()
plt.title('Allen/Altas Y Z scatter plot')    
plt.legend()